# Training: GDrive x BigQuery x GSheets
This notebook provides guidance on how you can connect your [Google Colab](https://colab.research.google.com/notebooks/welcome.ipynb#) instance to the following:

- Google Drive (GDrive) 
- BigQuery (BQ)
- Google Sheets (GSheets)

The interaction of these three is useful if you want to extract data from BigQuery and put it into a Google Sheet.

***

## Contents

The outline of this notebook is:

1. Mount GDrive
1. Connect to BigQuery
    + Import data from there
1. Prepare your data
1. Open and save into GSheets

***

## Set-up
We need to install and load the necessary packages and authenticate ourselves to use things like GDrive, BQ and GSheets.



In [0]:
# load packages
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

import pandas as pd
import re

import gspread
import gspread_dataframe as gd

***

## GDrive
We need to mount our GDrive to this session so we can access it. This location will be where we store the prepared data for other to use, a shared drive.

In [2]:
# mount GDrive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# show files in current directory
!ls

drive  sample_data


In [0]:
# navigate to folder of interest
%cd "drive/My Drive/Projects/training_googlecolaboratory"